In [1]:
import numpy as np
import mne
import pandas as pd
import sys

import csv
from scipy.stats import skew, kurtosis
import pyeeg as p
from numpy import nan
import math

In [2]:
def mean_variance(df):
    variance_vals = np.var(df)
    return np.mean(variance_vals)

In [3]:
def mean_kurtosis(df):
    kurtosis_vals = kurtosis(df)
    return np.mean(kurtosis_vals)

In [4]:
def mean_skewness(df):
    skew_vals = skew(df)
    return np.mean(skew_vals)

In [5]:
def mean_pfd(df):
    pfd_vals = []
    for col in df.columns:
        col = df[col].to_numpy()
        pfd_val = p.pfd(col)
        pfd_vals.append(pfd_val)
    return np.mean(pfd_vals)

In [6]:
def mean_hjorth_mob_comp(df):
    mob_vals = []
    comp_vals = []
    for col in df.columns:
        col = df[col].to_numpy()
        mob_col, comp_col = p.hjorth(col)
        mob_vals.append(mob_col)
        comp_vals.append(comp_col)
    return np.mean(mob_vals), np.mean(comp_vals)

In [7]:
def all_psd(data):
    fs = 256                                
    N = data.shape[1] # total num of points 

    # Get only in postive frequencies
    fft_vals = np.absolute(np.fft.rfft(data))

    n_rows = fft_vals.shape[0]
    n_cols = fft_vals.shape[1]
    psd_vals = np.zeros(shape=(n_rows, n_cols))

    for i in range(n_rows):
        for j in range(n_cols):
            psd_vals[i][j] = (N/fs) * fft_vals[i][j] * fft_vals[i][j];


    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(data.shape[1], 1.0/fs)

    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    psd_vals_list = []
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(psd_vals[:,freq_ix])
        psd_vals_list.append(eeg_band_fft[band] * 1000000)
    return psd_vals_list

In [8]:
def sum_psd(data):
    psd_vals = all_psd(data)
    return np.sum(psd_vals)

In [9]:
def mean_spectral_entropy(data):
    psd_vals = all_psd(data)
    power_ratio = []
    sum_psd_vals = sum_psd(data)
    for val in psd_vals:
        power_ratio.append(val/sum_psd_vals)
    bands = [0,4,8,12,30,45]
    Fs = 256
    spec_entropy = p.spectral_entropy(data, bands, Fs, power_ratio)
    return spec_entropy

In [10]:
def add_row(df_input, psd_ip, start, end, index):
    row_to_add = []
    d = df_input[index:index + duration]
    psd_ip = psd_ip[:, start:end]
    psd_ip = psd_ip[:][0]
    
    mean_var = mean_variance(d)
    mean_k = mean_kurtosis(d)
    mean_skew = mean_skewness(d)
    pfd = mean_pfd(d)
    h_mob, h_comp = mean_hjorth_mob_comp(d)
    mean_spec = mean_spectral_entropy(psd_ip)
    
    row_to_add.append(mean_var)
    row_to_add.append(mean_k)
    row_to_add.append(mean_skew)
    row_to_add.append(pfd)
    row_to_add.append(h_mob)
    row_to_add.append(h_comp)
    row_to_add.append(mean_spec)
    #Label: 1 = seizure, 0 = non-seizure. Change before running.
    row_to_add.append(1)
    
    
    return row_to_add

In [11]:
data = mne.io.read_raw_edf('chb01_04.edf')
header = ','.join(data.ch_names)
df = pd.DataFrame(data[:][0])
df = df.transpose()
# df = df.iloc[1467*256:1477*256]
# df

Extracting EDF parameters from C:\Users\upama\BTP\chb01_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-11-4f1ab8334ca3>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf('chb01_04.edf')


In [14]:
start = temp = 1457*256 
duration = 10*256
end = 1477*256

df1 = df.iloc[start:end, :]
df1 = pd.DataFrame(df1)
df1.shape

(5120, 23)

In [15]:
#processes one seizure in 10s windows
#adds rows with features extracted from these windows
# print(df1)
index = 0

res = pd.DataFrame()

#first iteration run in 'w' mode, all subsequent iteration run in 'a' mode
with open('test.csv', 'w') as file:
    writer = csv.writer(file)
    while temp < end:    
        row = add_row(df1, data, temp, temp + duration, index)
        res=res.append(pd.Series(row),ignore_index=True)
        writer.writerow(row)
        temp += duration
        index += duration

#res.columns = ['Variance', 'Kurtosis', 'Skewness', 'Petrosian Fractal Dimension', 'Hjorth Mobility', 'Hjorth Complexity', 'Spectral Entropy', 'Label']
res

,0,1,2,3,4,5,6,7
0,3.041031e-09,1.692586,0.003647,0.579843,0.004608,295.933273,0.438665,1.0
1,1.247285e-08,0.340726,-0.059379,0.568553,0.004571,354.352760,0.289082,1.0
